# GROUND STATE
Find the ground state of a spin glass system with nearest neighbour interactions

In [ ]:
import numpy as np
import itertools

from numba import jit
from spektral.utils.sparse import sp_matrix_to_sp_tensor

## Predefined Functions

In [ ]:
def Adj(D, L, sparse=False):
    N = L**D

    # create all nodes' coordinates
    nodes = [x for x in np.ndindex(tuple(np.repeat(L,D)))]

    # Pass from coordinate to node's index
    # (h,...k,j,i) <=> index = h*L^(D-1) + ... + k*L^2 + j*L + i
    mul = [L**i for i in reversed(range(D))]

    # Creation of adjacency matrix
    A_dense = []
    # creation of a row for each node's coordinate
    for node in nodes:
        temp_buffer = []
        A_dense_row = [0]*N
        # find the two nearest neighbours of the node along each dimension
        for d in range(D):
            temp=list(node)
            temp[d]=((temp[d]+1)%L)
            temp=np.inner(temp, mul)
            temp_buffer.append(temp)

            temp=list(node)
            temp[d]=((temp[d]-1)%L)
            temp=np.inner(temp, mul)
            temp_buffer.append(temp)

        temp_buffer=list(np.unique(np.array(temp_buffer), axis=0))
        for i in temp_buffer: A_dense_row[i]=1
        A_dense.append(A_dense_row)

    # sparse=False => sparse adjacency matrix
    # sparse=True => dense adjacency matrix
    if sparse:
        return sp_matrix_to_sp_tensor(np.array(A_dense))
    else:
        return np.array(A_dense)

In [ ]:
def J_inter(denseAdj):
    N = denseAdj.shape[0]
    sparseAdj = sp_matrix_to_sp_tensor(denseAdj)

    # sparse adjacency matrix as a numpy array
    edge=sparseAdj.indices.numpy()

    # ordered numpy sparse adjacency matrix
    un_edge=np.array([np.sort(i) for i in edge])

    # creation of the interaction array: (i,j) and (j,i) have the same Jij
    inter=[]
    for i in range(len(un_edge)):
        equal=True
        for j in range(i):
            if np.array_equal(un_edge[i],un_edge[j]):
                inter.append(inter[j])
                equal=False
                break
        if equal:
            inter.append(np.random.normal(0, 1))

    # creation of dense interaction matrix
    inter_matrix = np.zeros((N,N))
    counter = 0
    for i, j in edge:
        inter_matrix[i,j] = inter[counter]
        counter += 1
    return [np.array(inter).reshape(sparseAdj.indices.shape[0],1), inter_matrix.reshape((N,N,1))]

    # index of the returned list:
    # 0 => interaction array
    # 1 => interaction matrix (zero padded)

In [ ]:
@jit(nopython=True)
def computeEnergy(state, edge, interaction):
    energy = 0
    for i in range(len(edge)):
        energy -= interaction[i][0]*state[edge[i][0]][0]*state[edge[i][1]][0]
    return energy/2

In [ ]:
@jit(nopython=True)
def EnergyMinima(states, sparseAdj, inter):
    energy_min = np.inf
    for state in states:
        state = np.array(state).reshape(N,1)
        state_energy = computeEnergy(state, sparseAdj, inter)
        if state_energy<energy_min: energy_min=state_energy
    return energy_min

## Main

In [ ]:
D=2
L=3
N=L**D

sp_A = Adj(D, L, sparse=True).indices.numpy()
dense_A = Adj(D, L, sparse=False)
interaction = J_inter(dense_A)[0]

ensemble = [1, -1]
all_states = [x for x in itertools.product(ensemble, repeat=N)]

In [ ]:
energy_min = EnergyMinima(all_states, sp_A, interaction)

/root/venv/lib/python3.7/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'states' of function 'EnergyMinima'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-5-6d8e66e0d43e>", line 2:
@jit(nopython=True)
def EnergyMinima(states, sparseAdj, inter):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


In [ ]:
energy_min

-11.97939857066448